# NBA Machine Learning

The objective for our project will be to create a deep learning model that will be able to predict future outcomes of a basketball season. 
This is done because this could be used as a viable tool in the process of sports betting. The model would function almost exactly the same as one that predicts stock price and signals when to buy or sell, the only difference is that it would predict a player's stat and would signal when to bet instead of buying/selling.

Our project will use historical data from NBA teams such as previous season's records, shot making percentages, defensive ratings, 
and several others from the last ten seasons in a deep learning model to predict the next season's standings for each season and individual player statistics.


In [298]:
#import libraries required to run program
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [299]:
#import csv containing Jokic's statistics
jokic_csv_path = Path("Resources/jokic_2023.csv")
jokic_df = pd.read_csv(jokic_csv_path)
jokic_df.head()

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2022-10-19,27-242,DEN,@,UTA,L (-21),1,33:25,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2,2.0,2022-10-21,27-244,DEN,@,GSW,W (+5),1,34:24,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3,3.0,2022-10-22,27-245,DEN,NaN,OKC,W (+5),1,38:41,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4,4.0,2022-10-24,27-247,DEN,@,POR,L (-25),1,26:58,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5,5.0,2022-10-26,27-249,DEN,NaN,LAL,W (+11),1,34:48,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0


In [300]:
#remove rows with games Jokic did not play
jokic_df = jokic_df[jokic_df['GS'] != "Inactive"]

In [301]:
#dropped columns that were not relevant to model
jokic_df.drop(columns= ["Rk","Tm","Date","Age","Opp"], axis=1, inplace = True)
jokic_df.head()

,G,Unnamed: 5,Unnamed: 7,GS,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1.0,@,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2.0,@,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3.0,NaN,W (+5),1,38:41,6.0,10.0,0.600,1.0,1.0,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4.0,@,L (-25),1,26:58,3.0,4.0,0.750,0.0,0.0,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5.0,NaN,W (+11),1,34:48,12.0,17.0,0.706,0.0,4.0,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0


In [302]:
#added titles to columns without names
jokic_df.rename(columns={'Unnamed: 5':'Home_or_Away'},inplace= True)
jokic_df.rename(columns={'Unnamed: 7':'MOV'},inplace= True)

In [303]:
#changed Home or Away coumn so that if the game was at home it is a value of 0 and if away it is 1
jokic_df['Home_or_Away'] = jokic_df['Home_or_Away'].replace('@', 1)
jokic_df = jokic_df.fillna(0)

In [304]:
#convert minutes played column to seconds played 
def convert_to_numerical(time_obj):
    minutes, seconds = time_obj.split(':')
    total_seconds = int(minutes) * 60 + int(seconds)
    return total_seconds

jokic_df['MP'] = jokic_df['MP'].apply(convert_to_numerical)
jokic_df.rename(columns={'MP':'Seconds_played'},inplace =True)
jokic_df.head()

,G,Home_or_Away,MOV,GS,Seconds_played,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1.0,1.0,L (-21),1,2005,12.0,17.0,0.706,1.0,3.0,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2.0,1.0,W (+5),1,2064,7.0,13.0,0.538,1.0,2.0,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3.0,0.0,W (+5),1,2321,6.0,10.0,0.600,1.0,1.0,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4.0,1.0,L (-25),1,1618,3.0,4.0,0.750,0.0,0.0,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5.0,0.0,W (+11),1,2088,12.0,17.0,0.706,0.0,4.0,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0


In [305]:
#converted columns with float64 types to int64 types
jokic_int_df = jokic_df[['G', 'Home_or_Away', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']].round().astype('int64')


In [306]:
#concat dataframes that contained all columns
jokic_df.drop(columns= ['G', 'Home_or_Away', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'], axis=1, inplace = True)
jokic_full_df = pd.concat([jokic_df, jokic_int_df], axis=1)
jokic_full_df

,MOV,GS,Seconds_played,G,Home_or_Away,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,L (-21),1,2005,1,1,12,17,1,1,3,...,2,4,6,3,0,3,4,27,24,-5
1,W (+5),1,2064,2,1,7,13,1,1,2,...,9,12,10,0,0,5,4,26,25,-3
2,W (+5),1,2321,3,0,6,10,1,1,1,...,15,16,13,1,1,3,2,19,25,18
3,L (-25),1,1618,4,1,3,4,1,0,0,...,8,9,9,0,0,1,5,9,14,-10
4,W (+11),1,2088,5,0,12,17,1,0,4,...,12,13,9,4,0,3,3,31,34,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,W (+14),1,2016,65,1,12,16,1,2,3,...,7,12,7,3,0,2,1,31,36,28
73,W (+23),1,1913,66,0,10,20,0,1,6,...,5,6,11,1,0,3,2,31,28,19
74,W (+5),1,2098,67,0,8,11,1,0,0,...,14,17,12,0,2,3,3,25,32,11
78,L (-21),1,1481,68,1,7,13,1,0,2,...,8,10,4,2,3,8,1,14,10,-21


In [307]:
jokic_full_df.dtypes

MOV               object
GS                object
Seconds_played     int64
G                  int64
Home_or_Away       int64
FG                 int64
FGA                int64
FG%                int64
3P                 int64
3PA                int64
3P%                int64
FT                 int64
FTA                int64
FT%                int64
ORB                int64
DRB                int64
TRB                int64
AST                int64
STL                int64
BLK                int64
TOV                int64
PF                 int64
PTS                int64
GmSc               int64
+/-                int64
dtype: object